In [ ]:
!pip install torch transformers datasets accelerate -q

In [14]:
import torch
from transformers import pipeline, set_seed
from datasets import load_dataset
import random

# --- 1. Veri Setini Yükleme ve Hazırlama ---
print("--- 📚 Veri Seti Yükleniyor: CNN/DailyMail (Haber Makaleleri) ---")

# Hugging Face Datasets'ten CNN/DailyMail veri setini yükle
# "article" alanı haber makalesini içerir.
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:100]")  # İlk 100 örneği al
# Rastgele bir makale örneği seçelim
random_index = random.randint(0, len(dataset) - 1)
sample = dataset[random_index]

ARTICLE_INPUT = sample['article']
# Metin 512 token sınırını aşmaması için kısaltılabilir
ARTICLE_INPUT = ARTICLE_INPUT[:1500]

print(f"\n*** 📰 Seçilen Haber Metni (İlk 150 Karakter): ***\n{ARTICLE_INPUT[:150]}...\n")

# --- 2. T5: Makale Özetleme (Summarization) ---
def run_t5_summarization(text):
    """Verilen haber makalesini kısa ve tek cümlelik bir başlık özeti haline getirir."""
    print("\n--- 📑 T5: Makale Özetleme (Encoder-Decoder) ---")

    # T5 özetleme için iyi bir seçimdir.
    summarizer = pipeline('summarization', model='t5-small')  # t5-small daha hızlı

    # Özeti çalıştırma
    summary_result = summarizer(
        text,
        max_length=60,  # Kısa bir özet
        min_length=20,
        do_sample=False
    )[0]['summary_text']

    print(f"**T5 Özet:** {summary_result}")
    print("-" * 60)


# --- 3. BERT: Soru Cevaplama (Question Answering) ---
def run_bert_qa(text):
    """Makale metni içindeki bir soruya cevap bulur."""
    print("\n--- 🧠 BERT: Soru Cevaplama (Encoder) ---")

    # Soru Cevaplama (Question Answering) için optimize edilmiş bir model
    qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

    # Metne uygun bir soru tanımlayalım
    question = "What happened in this article?"

    # Cevabı çalıştırma
    result = qa_pipeline(question=question, context=text)

    print(f"Soru: {question}")
    print(f"Cevap: **{result['answer']}** (Güven: %{result['score']*100:.2f})")
    print("-" * 60)


# --- 4. GPT: Makaleyi Devam Ettirme (Generation) ---
def run_gpt_generation(text):
    """Makalenin sonuna bağlamsal olarak uygun bir cümle üretir."""
    print("\n--- 📝 GPT: Metin Devamı Üretimi (Decoder) ---")
    set_seed(42)  # Tekrarlanabilir sonuçlar için

    # Giriş olarak makalenin son 150 karakterini kullan
    prompt = text[-150:]

    # Metin Üretimi için pipeline oluşturma
    generator = pipeline('text-generation', model='gpt2')

    # Üretimi çalıştırma
    generated_text = generator(
        prompt,
        max_length=len(prompt.split()) + 30, # Mevcut metin + 30 token daha
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7 # Akıcı bir devam için
    )[0]['generated_text']

    # Üretilen metnin sadece eklenen kısmını göstermek için
    added_text = generated_text[len(prompt):].strip()

    print(f"Girdi (Makalenin sonu): ...{prompt.strip()}")
    print(f"**GPT Eklemesi (Devamı):** {added_text}")
    print("-" * 60)


# --- 5. Modelleri Çalıştır ---
# 1. T5'i Çalıştır (Özetleme)
run_t5_summarization(ARTICLE_INPUT)

# 2. BERT'ü Çalıştır (Soru Cevaplama)
# Not: Cevap makale içinde tam olarak geçmelidir.
run_bert_qa(ARTICLE_INPUT)

# 3. GPT'yi Çalıştır (Metin Üretme)
run_gpt_generation(ARTICLE_INPUT)

print("\n✅ Tüm model karşılaştırmaları tamamlandı!")


--- 📚 Veri Seti Yükleniyor: CNN/DailyMail (Haber Makaleleri) ---

*** 📰 Seçilen Haber Metni (İlk 150 Karakter): ***
(CNN)One hundred and forty-seven victims. Many more families affected. Even more broken hopes and dreams. As Kenyans mourned those killed last week in...


--- 📑 T5: Makale Özetleme (Encoder-Decoder) ---


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


**T5 Özet:** the hashtag #47notjustanumber is a reference to the number of people killed at garissa university . the attacks in Kenya killed 142 students, three security officers and two security personnel . many more families affected by the attacks .
------------------------------------------------------------

--- 🧠 BERT: Soru Cevaplama (Encoder) ---


Device set to use cuda:0


Soru: What happened in this article?
Cevap: **dreams died with his son** (Güven: %3.87)
------------------------------------------------------------

--- 📝 GPT: Metin Devamı Üretimi (Decoder) ---


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=56) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Girdi (Makalenin sonu): ...acks. The Interior Ministry has identified one of the attackers killed by security forces as the son of a government official. The father of suspect A
**GPT Eklemesi (Devamı):** imee Othman told Reuters that he had been staying in a hotel room in the vicinity of the attack when gunmen opened fire. Othman, a member of Egypt's security forces, was shot in the abdomen and was taken to a hospital for treatment. He was later released.

The attack comes amid a wave of violence in the country, at least two suicide bombings in Cairo and a stabbing attack in Cairo's Tahrir Square on Wednesday.

The attacks, which have killed more than 500 people, follow an uptick in attacks in recent weeks that have left more than 100 dead, according to security sources.

On Wednesday, al-Ansari, the youngest of the attackers, was shot and killed in western Cairo while leaving a party celebrating Eid al-Fitr, the Muslim holiday of remembrance, a security source said.

"We were there f